This is the test file for Porter's new models

To use it, change the modelPath variable to the path to the new model

Currently, it is configured for a model output of all letters, A-Z. If the model changes to exclude J and Z, please uncomment the line "if (index >= 9): index += 1" in the print section below. This should fix it for a 24 section output, removing J and Z. 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
cam_port = 0

cam = cv2.VideoCapture(cam_port)

workingDirectory = "../image_assets/"
imgPath = workingDirectory + 'frame256.png'
modelPath = '../models/COSC307_full_data_CNN.keras'

# Load the trained Keras model
model = load_model(modelPath)  # Replace with the path to your model file

valid, frame = cam.read()

frameSize = frame.shape
x = frameSize[1]
y = frameSize[0]

left = int((x - 256) / 2)
right = left + 256
top = int((y - 256) / 2)
bottom = top + 256


start_point = (left, top)    # top left corner of box
end_point = (right, bottom)      # bottom right corner of box
color = (0, 0, 255)         # red (B,G,R)
thickness = 2

# Function to preprocess the image
# Apply mean/std from training data
mean = 128.33125
std = 15.842568
def preprocess_image(img): # img is a np array (captured image)
    # Crop center 256x256 pixel array
    cropped = img[top:bottom, left:right]

    # Change to grayscale
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

    # Scale the image to (128, 128)
    resized = cv2.resize(gray, (0,0), fx = 0.5, fy = 0.5)

    # Scale pixel values to [-1, 1]
    normalized = (resized - mean) / std

    # Add batch size (1) and channel dimension (grayscale is 1 channel)
    processed_image = np.expand_dims(normalized, axis=(0, -1))  # Shape: (1, 128, 128, 1)

    return processed_image

counter = 0

while True:

    valid, frame = cam.read()   # frame holds the image object (np array)

    # rectangle is 256x256
    rectangleOverlay = cv2.rectangle(frame, start_point, end_point, color, thickness)
    cv2.imwrite(workingDirectory + "frameRect.png", rectangleOverlay)

    cv2.imshow("Current Frame", rectangleOverlay)

    if counter >= 25:
        counter = 0

        processed_image = preprocess_image(frame)

        prediction = model.predict(processed_image, batch_size=1)
        prediction = list(prediction[0])

        print("--------------------------------------")
        for i in range(5):
            maxProb = max(prediction)
            maxIndex = prediction.index(maxProb)
            index = maxIndex

            # if (index >= 9): index += 1
            print(f"Predicted Letter:  {chr(ord('A') + index)}, {int(maxProb * 100)}% chance")
            
            prediction[maxIndex] = 0
        print("--------------------------------------")
        
    
    else:
        counter += 1

    if cv2.waitKey(1) == ord('q'):
        break


cam.release()
cv2.destroyAllWindows()

The code below holds extra stuff that I may need if the model output changes

In [ ]:
# prediction = model.predict(processed_image, batch_size=1)

# probabilities = {}

# j = 0
# for i in range(26):
#     if i == 9 or i == 25:
#         continue
#     probabilities[prediction[0][i]] = chr(ord('A') + i)
#     j += 1

# ordered = sorted(probabilities, reverse=True)

# print("--------------------------------------")
# for i in range(5):
#     print(f"Predicted Letter:  {probabilities[ordered[i]]}, {int(ordered[i] * 100)}% chance")
# print("--------------------------------------")


# # new section


# prediction = model.predict(processed_image, batch_size=1)
# prediction = list(prediction[0])

# print("--------------------------------------")
# for i in range(5):
#     maxProb = max(prediction)
#     maxIndex = prediction.index(maxProb)
#     index = maxIndex

#     if (index >= 9): index += 1
#     print(f"Predicted Letter:  {chr(ord('A') + index)}, {int(maxProb * 100)}% chance")
    
#     prediction[maxIndex] = 0
# print("--------------------------------------")
